In [6]:
# get movie information from IMDb
from imdb import IMDb
ia = IMDb()
movie_list = []
for number in range(1, 20): # get first 19 movies
    movie_id = str(number).zfill(7)
    current_movie = ia.get_movie(movie_id)
    if len(current_movie) > 0: # if movie found by id
        movie_list.append(current_movie)
        
# write movie list into disk
import pickle
with open("test.txt", "wb") as myfile:
    pickle.dump(movie_list, myfile)

In [1]:
import httplib
import json

def tmdb_get_latest_movie():
    api_key = "7fbe755e9e83c66a573c6482cefd23ed"
    conn = httplib.HTTPSConnection("api.themoviedb.org")
    conn.request("GET", 
                 "/3/movie/latest?api_key="+api_key, 
                 "{}")
    return conn.getresponse().read().decode("utf-8")

def tmdb_get_movie_by_imdb_id(imdb_id):
    api_key = "7fbe755e9e83c66a573c6482cefd23ed"
    conn = httplib.HTTPSConnection("api.themoviedb.org")
    conn.request("GET", 
                 "/3/find/tt"+str(imdb_id)+"?api_key="+api_key+"&external_source=imdb_id", 
                 "{}")
    return conn.getresponse().read().decode("utf-8")

def tmdb_get_movie_by_tmdb_id(tmdb_id):
    api_key = "7fbe755e9e83c66a573c6482cefd23ed"
    conn = httplib.HTTPSConnection("api.themoviedb.org")
    conn.request("GET", 
                 "/3/movie/"+str(tmdb_id)+"?api_key="+api_key, 
                 "{}")
    return conn.getresponse().read().decode("utf-8")

def tmdb_get_adult(text):
    return json.loads(text)['adult']

def tmdb_get_collection(text):
    return json.loads(text)['belongs_to_collection']

def tmdb_get_budget(text):
    return json.loads(text)['budget']

def tmdb_get_genre(text):
    return json.loads(text)['genres']

def tmdb_get_tmdb_id(text):
    return json.loads(text)['id']

def tmdb_get_imdb_id(text):
    return json.loads(text)['imdb_id'][2::]

def tmdb_get_title(text):
    return json.loads(text)['title']

def tmdb_get_overview(text):
    return json.loads(text)['overview']

def tmdb_get_production_companies(text):
    return json.loads(text)['production_companies']

def tmdb_get_production_countries(text):
    return json.loads(text)['production_countries']

def tmdb_get_release_date(text):
    return json.loads(text)['release_date']

def tmdb_get_runtime(text):
    return json.loads(text)['runtime']

def tmdb_get_spoken_languages(text):
    return json.loads(text)['spoken_languages']

def tmdb_get_poster_url(text):
    return "https://image.tmdb.org/t/p/w500" + json.loads(text[1:-5])['poster_path']

def tmdb_exist(text):
    error_msg = "The resource you requested could not be found."
    if error_msg in text:
        return False
    else:
        return True

In [2]:
# sample movie TMDb id
movie_tmdb_id = 315837

# get movie data text
text = tmdb_get_movie_by_tmdb_id(movie_tmdb_id)

print "movie title:", tmdb_get_title(text)
print "IMDb id:", tmdb_get_imdb_id(text)
print "TMDb id:", tmdb_get_tmdb_id(text)
print "genre:", tmdb_get_genre(text)
print "movie poster:"
poster_url = tmdb_get_poster_url(text)

# get poster from url
io.imshow(io.imread(poster_url))
io.show()

movie title: Ghost in the Shell
IMDb id: 1219827
TMDb id: 315837
genre: [{u'id': 28, u'name': u'Action'}, {u'id': 18, u'name': u'Drama'}, {u'id': 878, u'name': u'Science Fiction'}]
movie poster:


ValueError: Extra data: line 1 column 8 - line 1 column 1086 (char 7 - 1085)

In [ ]:
import time
import sys
import os

dir_name = os.path.expanduser('~/tmdb_movie/')
if not os.path.exists(dir_name):
    os.makedirs(dir_name)

latest_movie_text = tmdb_get_latest_movie()
latest_movie_tmdb_id = tmdb_get_tmdb_id(latest_movie_text)
time.sleep(0.26)

for current_id in range(72961, int(latest_movie_tmdb_id)+1):
    current_id_string = str(current_id).zfill(6)
    current_movie_text = tmdb_get_movie_by_tmdb_id(current_id_string)
    time.sleep(0.26)
    if tmdb_exist(current_movie_text):
        file_name = "tmdb_"+current_id_string+".txt"
        full_path = os.path.join(dir_name, file_name)
        with open(full_path, "wb") as myfile:
            pickle.dump(current_movie_text, myfile)
    print "\rdownloaded movie id {}/{}, movie exist? {}".format(current_id_string, 
                                                                latest_movie_tmdb_id,
                                                               tmdb_exist(current_movie_text)),
    sys.stdout.flush()
    

downloaded movie id 073140/450491, movie exist? True